# P1

## (a)

### 1.Bayesian Regression

In [12]:
def BR(x, y, n, q, prior1, prior2):
    A = [[0.0 for _ in range(n)] for _ in range(q + 1)]

    for i in range(n):
        for j in range(q + 1):
            A[j][i] = x[i] ** j

    A_T = [[A[j][i] for j in range(q + 1)] for i in range(n)]
    
    ATA = [[0.0 for _ in range(q + 1)] for _ in range(q + 1)]
    for i in range(q + 1):
        for j in range(q + 1):
            ATA[i][j] = sum(A[i][k] * A[j][k] for k in range(n))
    
    for i in range(q + 1):
        ATA[i][i] += 1 / prior1

    sigma = invert_matrix(ATA)

    Ab = [0.0 for _ in range(q + 1)]
    for i in range(q + 1):
        Ab[i] = sum(A[i][k] * y[k] for k in range(n))
    
    miu = [0.0 for _ in range(q + 1)]
    for i in range(q + 1):
        miu[i] = (1 / prior2) * sum(sigma[i][j] * Ab[j] for j in range(q + 1))
    
    return miu, sigma

### 2.LASSO

In [15]:
def LASSO(x, y, n, q):
    b = [0] * n
    A = [[0] * (q + 1) for _ in range(n)]  # Create an empty matrix A of size n x (q+1)

    for i in range(n):
        b[i] = y[i]
        for j in range(q + 1):
            A[i][j] = x[i] ** j

    lamda = 1.8

    H = [[0] * (2 * q + 2) for _ in range(2 * q + 2)]
    f = [0] * (2 * q + 2)

    ATA = [[0] * (q + 1) for _ in range(q + 1)]
    for i in range(n):
        for j in range(q + 1):
            for k in range(q + 1):
                ATA[j][k] += A[i][j] * A[i][k]

    for i in range(q + 1):
        for j in range(q + 1):
            H[i][j] = ATA[i][j]
            H[i + q + 1][j + q + 1] = ATA[i][j]
            H[i][j + q + 1] = -ATA[i][j]
            H[i + q + 1][j] = -ATA[i][j]

    Ab = [0] * (q + 1)
    for i in range(n):
        for j in range(q + 1):
            Ab[j] += A[i][j] * b[i]

    for i in range(q + 1):
        f[i] = lamda - Ab[i]
        f[i + q + 1] = lamda - Ab[i]
    return None


### 3.Linear Square

In [18]:
def LS(x, y, n, q):
    b = [0] * n
    A = [[0] * (q + 1) for _ in range(n)]

    for i in range(n):
        b[i] = y[i]
        for j in range(q + 1):
            A[i][j] = x[i] ** j

    ATA = [[0] * (q + 1) for _ in range(q + 1)]
    
    for i in range(n):
        for j in range(q + 1):
            for k in range(q + 1):
                ATA[j][k] += A[i][j] * A[i][k]

    Ab = [0] * (q + 1)
    for i in range(n):
        for j in range(q + 1):
            Ab[j] += A[i][j] * b[i]

    def gauss_jordan(matrix):
        n = len(matrix)
        augmented = [row + [1 if i == j else 0 for j in range(n)] for i, row in enumerate(matrix)]
        
        for i in range(n):
            max_row = max(range(i, n), key=lambda r: abs(augmented[r][i]))
            augmented[i], augmented[max_row] = augmented[max_row], augmented[i]
            pivot = augmented[i][i]
            for j in range(2 * n):
                augmented[i][j] /= pivot
            
            for j in range(n):
                if j != i:
                    factor = augmented[j][i]
                    for k in range(2 * n):
                        augmented[j][k] -= factor * augmented[i][k]

        inverse = [row[n:] for row in augmented]
        return inverse

    ATA_inv = gauss_jordan(ATA)
    theta = [0] * (q + 1)
    for i in range(q + 1):
        for j in range(q + 1):
            theta[i] += ATA_inv[i][j] * Ab[j]

    return theta


### 4.Regularized LS

In [21]:
def RLS(x, y, n, q):
    A = [[0.0 for _ in range(n)] for _ in range(q + 1)]
    
    for i in range(n):
        for j in range(q + 1):
            A[j][i] = x[i] ** j

    A_T = [[A[j][i] for j in range(q + 1)] for i in range(n)]
    
    ATA = [[0.0 for _ in range(q + 1)] for _ in range(q + 1)]
    for i in range(q + 1):
        for j in range(q + 1):
            ATA[i][j] = sum(A[i][k] * A[j][k] for k in range(n))
    
    lamda = 10
    Lambda_I = [[lamda if i == j else 0.0 for j in range(q + 1)] for i in range(q + 1)]
    
    ATA_Lambda_I = [[ATA[i][j] + Lambda_I[i][j] for j in range(q + 1)] for i in range(q + 1)]
    ATA_Lambda_I_inv = invert_matrix(ATA_Lambda_I)
    
    Ab = [0.0 for _ in range(q + 1)]
    for i in range(q + 1):
        Ab[i] = sum(A[i][k] * y[k] for k in range(n))
    theta = [0.0 for _ in range(q + 1)]
    for i in range(q + 1):
        theta[i] = sum(ATA_Lambda_I_inv[i][j] * Ab[j] for j in range(q + 1))
    
    return theta


def invert_matrix(matrix):
    size = len(matrix)
    identity = [[1.0 if i == j else 0.0 for j in range(size)] for i in range(size)]
    for i in range(size):
        diag_element = matrix[i][i]
        for j in range(size):
            matrix[i][j] /= diag_element
            identity[i][j] /= diag_element
        for k in range(size):
            if k != i:
                factor = matrix[k][i]
                for j in range(size):
                    matrix[k][j] -= factor * matrix[i][j]
                    identity[k][j] -= factor * identity[i][j]
    return identity

### 5.Robust Regression

In [26]:

from scipy.optimize import linprog
import numpy as np

def robust(x, y, n, q):
    b = np.zeros(n)
    A = np.zeros((q + 1, n))
    
    for i in range(n):
        b[i] = y[i]
        for j in range(q + 1):
            A[j, i] = x[i] ** (j)
    
    f = np.concatenate([np.zeros(q + 1), np.ones(n)])
    B = np.vstack([
        np.hstack([-A.T, -np.eye(n)]),
        np.hstack([A.T, -np.eye(n)])
    ])
    
    c = np.concatenate([-b, b])
    res = linprog(f, A_ub=B, b_ub=c, method='highs')

    if res.success:
        theta = res.x[:q+1]
    return theta

## (b)

|  LS  |  RLS(0.48)  |  LASSO(0)  |  RR  |  BR  |
| :--- | :---------- | :--------- | :--- | :--- |
|  0.4086  |  0.4076  |  0.4086  |  0.7680  |  0.4592  |

## (c)

|  subset  |  LS  |  RLS(0.48)  |  LASSO(0.5)  |  RR  |  BR  |
|  :-----  | :--- | :---------- | :--------- | :--- | :--- |
|  10%  |  28.312  |  82.205  |  27.406  |  13.0697  |  31.689  |
|  25%  |  10.793  |  3.475  |  1.138  |  1.011  |  13.292  |
|  50%  |  12.857  |  0.284  |  0.950  |  3.950  |  0.307  |
|  75%  |  0.332  |  0.941  |  0.563  |  0.257  |  0.730  |

## (d)

|  LS  |  RLS(0.48)  |  LASSO(1)  |  RR  |  BR  |
| :--- | :---------- | :--------- | :--- | :--- |
|  2.746  |  2.352  |  2.551  |  0.933  |  1.660  |

## (e)

|  LS  |  RLS(0.48)  |  LASSO(1)  |  RR  |  BR  |
| :--- | :---------- | :--------- | :--- | :--- |
|  7.983  |  2.877  |  2.637  |  1.289  |  3.043  |

# P2

## (a)

|      |  LS  |  RLS(0.68)  |  LASSO(3)  |  RR  |  BR  |
| :--- | :--- | :---------- | :--------- | :--- | :--- |
|  MSE  |  3.102  |  2.620  |  2.451  |  13.112  |  3.146  |
|  MAE  |  1.365  |  1.277  |  1.250  |  1.365  |  1.448  |

## (b)

|      |  LS  |  RLS(0.56)  |  LASSO(3)  |  RR  |  BR  |
| :--- | :--- | :---------- | :--------- | :--- | :--- |
|  MSE  |  2.924  |  2.425  |  2.265  |  2.898  |  2.913 |
|  MAE  |  1.327  |  1.208 |  1.177 |  1.307 |  1.297  |